In [98]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import warnings

warnings.filterwarnings("ignore")

# Nettoyage des données

In [21]:
df1 = pd.read_csv('fr.openfoodfacts.org.products.csv', sep='\t')

C:\Users\nberraba\AppData\Local\Temp\ipykernel_8820\511897635.py:1: DtypeWarning: Columns (0,3,5,19,20,24,25,26,27,28,35,36,37,38,39,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('fr.openfoodfacts.org.products.csv', sep='\t')


In [ ]:
df1['pnns_groups_1']

In [33]:
dictio = {i : df1[['nutrition-score-fr_100g', 'nutrition-score-uk_100g']][df1['main_category_fr'] == i].median() for i in df1['main_category_fr'].unique()}

In [43]:
dictio

{nan: nutrition-score-fr_100g   NaN
 nutrition-score-uk_100g   NaN
 dtype: float64,
 'Filet-de-boeuf': nutrition-score-fr_100g   NaN
 nutrition-score-uk_100g   NaN
 dtype: float64,
 'Légumes frais': nutrition-score-fr_100g   -6.0
 nutrition-score-uk_100g   -6.0
 dtype: float64,
 'Pâtisseries': nutrition-score-fr_100g    17.0
 nutrition-score-uk_100g    17.0
 dtype: float64,
 'Pains': nutrition-score-fr_100g    1.0
 nutrition-score-uk_100g    1.0
 dtype: float64,
 'Boulange': nutrition-score-fr_100g    1.0
 nutrition-score-uk_100g    1.0
 dtype: float64,
 'Aliments et boissons à base de végétaux': nutrition-score-fr_100g   -1.0
 nutrition-score-uk_100g   -1.0
 dtype: float64,
 'Root-bier': nutrition-score-fr_100g   NaN
 nutrition-score-uk_100g   NaN
 dtype: float64,
 'Sablés': nutrition-score-fr_100g    20.0
 nutrition-score-uk_100g    20.0
 dtype: float64,
 'Plats préparés': nutrition-score-fr_100g    2.0
 nutrition-score-uk_100g    2.0
 dtype: float64,
 'Volailles': nutrition-score-fr

In [ ]:
 myDict =  {i: df[liste_colonnes][df['main_category_fr'] == i].median() for i in df['main_category_fr'].unique()}
    
    for i in df4['main_category_fr'].unique():
        df.loc[df['main_category_fr']== i, liste_colonnes] = df[df['main_category_fr']== i ][liste_colonnes].fillna(df[df['main_category_fr'] == i][liste_colonnes].median()) 
    

In [99]:
#fonction qui importe un fichier csv et renvoie un dataframe

def importer_csv(nom_fichier, chemin):
    chemin_complet = chemin  + nom_fichier 
    dataframe = pd.read_csv(chemin_complet, sep='\t')
    return dataframe

In [100]:
#fonction qui supprime les colonnes vides et des lignes doublons

def supprimer_col_vides_doublons(df):
    df.dropna(axis=1, how='all', inplace=True)
    df.drop_duplicates()
    
    return df

In [101]:
def suppression_col_correlees(df, seuil_correlation):
    #création d'un dataframe quantitatif
    df_quantitatif = pd.DataFrame()

    for colonne in df.columns:
        if pd.api.types.is_numeric_dtype(df[colonne]):
            df_quantitatif[colonne] = df[colonne]
            
    # Calculer la matrice de corrélation
    matrice_correlation = df_quantitatif.corr()

    # Identifier les paires de colonnes corrélées
    colonnes_correlees = set()
    for i in range(len(matrice_correlation.columns)):
        for j in range(i):
            if abs(matrice_correlation.iloc[i, j]) >= seuil_correlation:
                colonne_i = matrice_correlation.columns[i]
                colonne_j = matrice_correlation.columns[j]
                colonnes_correlees.add((colonne_i, colonne_j))

     # Supprimer la première colonne de chaque paire corrélée
    for colonne1, colonne2 in colonnes_correlees:
        df.drop(columns=colonne1, inplace=True)
    
    return df

In [102]:
#fonction qui ne garde que les colonnes pertinentes

def selection_colonnes(df):
    #creation de la liste des variables, on y insérant des variables utiles vis à vis de l'analyse
    variables = ['product_name', 'brands','pnns_groups_1', 'pnns_groups_2', 'main_category_fr', 'nutrition_grade_fr']
    
    #liste de toutes les colonnes en rapport avec les valeurs nutritionnelles
    val_nutri = [col for col in df.columns if col.endswith('_100g')]
    
    #suppression de l'empreinte carbone qui n'affecte pas la valeur nutritionelle, ainsi que le nutriscore UK
    try:
        val_nutri.remove('carbon-footprint_100g')
    except KeyError:
        print('colonne inexistente')
    try:
        val_nutri.remove('nutrition-score-uk_100g')
    except KeyError:
        print('colonne inexistente')
        
    variables = variables + val_nutri
    
    #ne garder que ces variables dans le dataframe
    df = df[variables]
    #liste des colonnes avec +60% de valeurs nulles
    colonnes_a_supprimer = df[val_nutri].columns[round(df[val_nutri].isna().mean()*100,1)> 60]
    
    #suppression de ces colonnes vides ou presque
    df = df.drop(columns=colonnes_a_supprimer)
    
    #suppression d'une colonne si +0.85 de corrélation avec une autre
    df = suppression_col_correlees(df, 0.85)
    
    return df



In [103]:
def remplacer_na_100g_mediane(df, liste_colonnes):
    myDict =  {i: df[liste_colonnes][df['main_category_fr'] == i].median() for i in df['main_category_fr'].unique()}
    
    for i in df['main_category_fr'].unique():
        df.loc[df['main_category_fr']== i, liste_colonnes] = df[df['main_category_fr']== i ][liste_colonnes].fillna(df[df['main_category_fr'] == i][liste_colonnes].median()) 
    
    return df

In [104]:
def gerer_valeurs_manquantes(df):
    
    #suppression des lignes où le nom du produit n'est pas renseigné
    df = df.dropna(subset=['product_name'])
    
    col_100g = [col for col in df.columns if col.endswith('_100g')]
    
    #remplacer des Nan par la médiane de la colonne
    df = remplacer_na_100g_mediane(df, col_100g)
    
    return df

In [105]:
def valeurs_aberrantes(df):
    df = df[(df['nutrition-score-fr_100g'] >= -15) & (df['nutrition-score-fr_100g'] <= 40)]
    
    df = df[(df['energy_100g'] < 4000)]
    
    # Sélectionner les colonnes finissant par "100g" sauf "energy_100g"
    colonnes_a_selectionner = [col for col in df.columns if col.endswith('100g') and col != 'energy_100g' 
                               and col != 'nutrition-score-fr_100g']
    
    df_val_nutri = df[colonnes_a_selectionner]
    
    mask = ((df_val_nutri > 100) | (df_val_nutri < 0)).any(axis=1)
    
    df.drop(index=df[mask].index, inplace=True)
    return df

In [106]:
#défnition variables

chemin = ''
fichier = 'fr.openfoodfacts.org.products.csv'
nom_csv_export = 'sante_publique.csv'

In [107]:
def main():
    import_librairies()
    df = importer_csv(fichier, chemin)
    df = supprimer_col_vides_doublons(df)
    df = selection_colonnes(df)
    df = gerer_valeurs_manquantes(df)
    df = valeurs_aberrantes(df)
    print(df.shape)
    
    # Exporter le DataFrame dans un fichier CSV
    df.to_csv(chemin + nom_csv_export, index=False)

## Main

In [108]:
main()

Index(['product_name', 'brands', 'pnns_groups_1', 'pnns_groups_2',
       'main_category_fr', 'nutrition_grade_fr', 'energy_100g', 'fat_100g',
       'saturated-fat_100g', 'trans-fat_100g', 'cholesterol_100g',
       'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g',
       'salt_100g', 'vitamin-a_100g', 'vitamin-c_100g', 'calcium_100g',
       'iron_100g', 'nutrition-score-fr_100g'],
      dtype='object')
(242437, 21)


# Analyse

In [ ]:
df

In [ ]:
myDict =  {i: df4[col_100g][df4['main_category_fr'] == i].mean() for i in df4['main_category_fr'].unique()}

In [ ]:
for i in df4['main_category_fr'].unique():
    
    df4.loc[df4['main_category_fr']== i, col_100g] = df4[df4['main_category_fr']== i ][col_100g].fillna(df4[df4['main_category_fr'] == i][col_100g].mean()) 